### 상세 페이지에서 크롤링 할 정보

1. 스냅샷 
<img src="https://image.musinsa.com/mfile_s01/_street_images/51263/800.street_5c94a6a2d06aa.jpg?20190322182728" width=400>
2. 관련 정보 
<img src="https://www.dropbox.com/s/3edp92aczpiiwhl/Screenshot%202019-03-25%2013.00.28.png?raw=1" width=400>
3. 연결된 제품 
<img src="https://www.dropbox.com/s/qhkjwieb5fzjue6/Screenshot%202019-03-25%2013.01.31.png?raw=1" width=400>

In [6]:
import requests 
from bs4 import BeautifulSoup
import os

from sub_crawler import *

In [3]:
!pip install selenium

    100% |████████████████████████████████| 911kB 8.5MB/s eta 0:00:01


In [5]:
def login_info():

    id_value = str(input())
    pw_value = str(input())
    
    return {"id" : id_value, "pw": pw_value}    

def page_scrapper(page_number):
    """
    scrapping a given page 
    
    argument : 해당 페이지 번호
    
    return : 페이지 내의 사진 엘리먼트 리스트  
    """
    global info 
    info = login_info()
    url = "https://www.musinsa.com/index.php?m=street&_mon=&p={}#listStart".format(page_number)
    res = requests.session().post(url, data=info) 
    # raise error if response != 200 
    res.raise_for_status() 
    return BeautifulSoup(res.content, 'html.parser').select("div .articleImg")
    
    
def pic_link_parser(picture):
    picture.select_one("a")['href'].strip()
    base_url = "https://www.musinsa.com"
    return base_url + picture.select_one("a")['href'].strip()


def image_download(download_link, title, path):
    
    download_link = "http:" + download_link
    res = requests.get(download_link, stream=True)
    path += "/{}".format(title)
    
    with open(path, 'wb') as f:
        
        for chunk in res.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)
    return

In [5]:
page_number = 1 

pages = page_scrapper(page_number)
spec_urls = list(map(pic_link_parser, pages))

donotlose89
anfkzkal8992


### 하나의 사진에 연결된 아이템 데이터

In [ ]:
def get_styleInfo(soup):
    """
    스냅샷의 정보 와 관련 태그 
    """
    values = list(map(lambda x : x.text, soup.select_one("table").select("td > span")))
    keys = list(map(lambda x : x.text, soup.select_one("table").select("th > span")))
    
    related_tags = list(map(lambda x : x.text, soup.select_one("table").select(".listItem span")))
    
    infoDict = {k : v for k, v in zip(keys, values)}
    
    return infoDict, related_tags

def Item_spec(spec_url):
    """
    하나의 스냅샷 상세페이지에서 해당 사진과 연관된 패션아이템을 크롤링

    """
    res = requests.session().post(spec_url, data=info)
    soup = BeautifulSoup(res.content, 'html.parser')
    # 스냅샷 정보 
    style_info, related_tags = get_styleInfo(soup)
    # 스냅샷 이미지 
    soup.select_one("div > .styleItem-list").select(".itemImg")
    

In [ ]:
# snap shot 

In [47]:
# 세부 정보 

In [75]:
# 
soup.select_one("div > .styleItem-list").select(".itemImg") # 이 것의 갯수가 0이면 넘어가는 조건 문 

[<div class="itemImg">
 <a class="lbox img_st_view2" href="//image.musinsa.com/mfile_s01/_street_images/51263/800.street_img_view15c94a6af8d657.jpg?20190322182728" title="동대문_김희연_9S Piping Hood Jumpe-메인부스"><img alt="동대문_김희연_9S Piping Hood Jumpe-메인부스" src="//image.musinsa.com/mfile_s01/_street_images/51263/280.street_img_view15c94a6af8d657.jpg?20190322182728"/></a>
 <div class="itemInfo">
 <p class="title">
 										메인부스의 9S 피핑 후드 점퍼									</p>
 <ul class="article-tag-list list">
 <li class="listItem"><a href="/?m=street&amp;tag=9S Piping Hood Jumpe"><span>9S Piping Hood Jumpe</span></a></li><li class="listItem"><a href="/?m=street&amp;tag=메인부스"><span>메인부스</span></a></li>
 <li class="listItem tagGray"><a href="/?m=street&amp;ptype=2"><span>아우터</span></a></li><li class="listItem tagGray"><a href="/?m=street&amp;color=6"><span>녹색</span></a></li>
 </ul>
 </div>
 </div>, <div class="itemImg">
 <a class="lbox img_st_view2" href="//image.musinsa.com/mfile_s01/_street_images/51263/800.stre

In [107]:
# img link 
soup.select_one("div > .styleItem-list").select(".itemImg")[0].select("a")[0]["href"]

'//image.musinsa.com/mfile_s01/_street_images/51263/800.street_img_view15c94a6af8d657.jpg?20190322182728'

In [112]:
# product title 
soup.select_one("div > .styleItem-list").select(".itemImg")[0].select('p')[0].text.strip()

'메인부스의 9S 피핑 후드 점퍼'

In [116]:
# tag list
list(map(lambda x : x.text, soup.select_one("div > .styleItem-list").select(".itemImg")[0].select("ul > li")))

['9S Piping Hood Jumpe', '메인부스', '아우터', '녹색']

In [ ]:
# selenium

In [117]:
spec_urls[0]

'https://www.musinsa.com/index.php?m=street&p=1&uid=51263'